In [22]:
import csv
import re
import numpy as np

param_re = re.compile(r"ch(\d+)=(\-?\d+\.?\d*e?[+\-]?\d*)", re.IGNORECASE)

def parse_cadence_csv(path):
    """
    Returns:
      challenges: np.ndarray (256, 8) with voltages (ch0..ch7)
      diffs:      np.ndarray (256,) with diff values
      bits:       np.ndarray (256,) with 0/1 (1 if diff>0)
    """
    challenges = []
    diffs = []

    with open(path, newline='') as f:
        reader = csv.reader(f)
        current_ch = None
        for row in reader:
            if not row:
                continue
            # Parameters row
            if row[0].startswith("Parameters:"):
                line = row[0]
                vals = {int(m.group(1)): float(m.group(2))
                        for m in param_re.finditer(line)}
                current_ch = [vals[i] for i in range(8)]  # ch0..ch7
            # Data row
            elif row[1:3] == ['base_setup', 'diff'] and row[3] != '':
                diff = float(row[3])
                if current_ch is None:
                    continue
                challenges.append(current_ch)
                diffs.append(diff)

    challenges = np.array(challenges)          # (N,8)
    diffs = np.array(diffs)                    # (N,)
    bits = (diffs > 0).astype(int)
    return challenges, diffs, bits

challenges, diffs, outputs = parse_cadence_csv("data/onlydiff.csv")


In [23]:
print(challenges.shape)  
print(outputs.shape)

(256, 8)
(256,)


In [24]:
import numpy as np

# challenges : (256, 8)
# outputs    : (256,)

uniformity = outputs.mean()
print("Uniformity (fraction of 1s):", uniformity)

Uniformity (fraction of 1s): 0.4921875
